<a href="https://colab.research.google.com/github/creatorArjun/NASA_project/blob/test-branch/notebookdb68ea80bb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'space-apps-2024-seismic-detection:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5700575%2F9436224%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241005%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241005T105615Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D8a5d9a0e0c18d7cf0b91f5e71de7f7e9948df0474b98d2e34ca373dae90cc251431777f62f48fe73e240223222ea677389359d141b2472d0e2f47cc0d18c1bba7f7011d1a974339a2c8d40d4a171f0de6d1772542392d44d2ff912697bdc0912872a35b6364643a55c72d5f199de017ff9cb65450fea713af06dd92642f902692335876285224f41ed623fcc18397d337f9505fb169f1ac6804f5b5525d5811aeb4a2e949e6a977b80234075c771d254de1e758b4bf125ac8c78c5b438aef65499d5854e8fd8424b3c16e8c12fa217ad0983bc3e187b60a32f9becca92b85b00731720aa808e6cdef5bf110e08c16e671cb22ec567ea99619bce94e6afa6170f,mseed-lunar:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5820545%2F9552688%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241005%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241005T105615Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D65e89481ff1ba048e4f79998154a1de0784a33fd93fab5ff9c3aa33659667d5ce14136a8f7814b872777c28ea4373c1ec6280a9863506a925fa31d5b081c5d4387e5be01ca7f210ad66488bc16e447fa271492db123200d94b2bd68ab68824eec9961125d96b568fcc4182449fd29b493b50a222b849eda7f567e20157a97707e7f4898434bc3d5eaa0ba53bc46526e24e7d3885af57aa73e95be8ea0714b188a842c65ffdeaef30a5cd5f454b82d064ca3e0307560ed676174db16dc8c27b60fae991d6927b47c674ae7458fdac893a684fdcb6b0f77e1bdcc002961637ea979f0d439384779c7fbb948a33c714bd98acb986e13030d05225b0bbccf82ba676'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 2479275106 bytes downloaded
Downloaded and uncompressed: space-apps-2024-seismic-detection
[==================================================] 758102366 bytes downloaded
Downloaded and uncompressed: mseed-lunar
Data source import complete.


In [1]:
!pip install obspy pandas

import obspy
import pandas as pd

# Load the seismic data
stream = obspy.read('/kaggle/input/mseed-lunar/xa.s12.00.mhz.1969-12-16HR00_evid00006.mseed')

# Load the catalog (assuming it's a CSV file)
catalog = pd.read_csv('/kaggle/input/mseed-lunar/apollo12_catalog_GradeA_final.csv')


In [2]:
def calculate_sta_lta(trace, nsta, nlta):
    """
    Calculate the STA/LTA ratio for a given seismic trace.
    nsta: number of samples in short-term window
    nlta: number of samples in long-term window
    """
    cft = obspy.signal.trigger.classic_sta_lta(trace.data, nsta, nlta)
    return cft

# Example usage for a single trace
trace = stream[0]  # Select the first trace from the stream
nsta = int(0.5 * trace.stats.sampling_rate)  # Short-term window (~0.5 seconds)
nlta = int(5.0 * trace.stats.sampling_rate)  # Long-term window (~5 seconds)
cft = calculate_sta_lta(trace, nsta, nlta)


AttributeError: module 'obspy' has no attribute 'signal'

In [3]:
!pip install obspy==1.3.0
#The current version of obspy has moved the signal processing functions.
#Installing obspy 1.3.0 will fix this issue

import obspy
import pandas as pd

# Load the seismic data
stream = obspy.read('/kaggle/input/mseed-lunar/xa.s12.00.mhz.1969-12-16HR00_evid00006.mseed')

# Load the catalog (assuming it's a CSV file)
catalog = pd.read_csv('/kaggle/input/mseed-lunar/apollo12_catalog_GradeA_final.csv')


def calculate_sta_lta(trace, nsta, nlta):
    """
    Calculate the STA/LTA ratio for a given seismic trace.
    nsta: number of samples in short-term window
    nlta: number of samples in long-term window
    """
    cft = obspy.signal.trigger.classic_sta_lta(trace.data, nsta, nlta)
    return cft

# Example usage for a single trace
trace = stream[0]  # Select the first trace from the stream
nsta = int(0.5 * trace.stats.sampling_rate)  # Short-term window (~0.5 seconds)
nlta = int(5.0 * trace.stats.sampling_rate)  # Long-term window (~5 seconds)
cft = calculate_sta_lta(trace, nsta, nlta)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 3.8 MB/s eta 0:00:00
  Attempting uninstall: obspy
    Found existing installation: obspy 1.4.1
    Uninstalling obspy-1.4.1:
      Successfully uninstalled obspy-1.4.1


AttributeError: module 'obspy' has no attribute 'signal'

In [1]:
!pip install obspy==1.3.0
import os
import obspy
import pandas as pd

def calculate_sta_lta(trace, nsta, nlta):
  """
  Calculate the STA/LTA ratio for a given seismic trace.
  nsta: number of samples in short-term window
  nlta: number of samples in long-term window
  """
  cft = obspy.signal.trigger.classic_sta_lta(trace.data, nsta, nlta)
  return cft

def process_mseed_file(file_path):
  """
  Loads an mseed file, calculates STA/LTA, and returns the result.
  """
  try:
    stream = obspy.read(file_path)
    trace = stream[0]
    nsta = int(0.5 * trace.stats.sampling_rate)
    nlta = int(5.0 * trace.stats.sampling_rate)
    cft = calculate_sta_lta(trace, nsta, nlta)
    return cft
  except Exception as e:
    print(f"Error processing {file_path}: {e}")
    return None

# Directory containing your mseed files
mseed_directory = '/kaggle/input/mseed-lunar'

results = []
for filename in os.listdir(mseed_directory):
  if filename.endswith(".mseed"):
    file_path = os.path.join(mseed_directory, filename)
    cft = process_mseed_file(file_path)
    if cft is not None:
      results.append({'filename': filename, 'cft': cft})

# Create a Pandas DataFrame from the results
results_df = pd.DataFrame(results)
print(results_df)

Error processing /kaggle/input/mseed-lunar/xa.s12.00.mhz.1973-08-08HR00_evid00437.mseed: module 'obspy' has no attribute 'signal'
Error processing /kaggle/input/mseed-lunar/xa.s12.00.mhz.1971-10-31HR00_evid00045.mseed: module 'obspy' has no attribute 'signal'
Error processing /kaggle/input/mseed-lunar/xa.s12.00.mhz.1971-05-13HR00_evid00033.mseed: module 'obspy' has no attribute 'signal'
Error processing /kaggle/input/mseed-lunar/xa.s12.00.mhz.1974-03-25HR00_evid00140.mseed: module 'obspy' has no attribute 'signal'
Error processing /kaggle/input/mseed-lunar/xa.s12.00.mhz.1970-10-24HR00_evid00014.mseed: module 'obspy' has no attribute 'signal'
Error processing /kaggle/input/mseed-lunar/xa.s12.00.mhz.1970-06-26HR00_evid00009.mseed: module 'obspy' has no attribute 'signal'
Error processing /kaggle/input/mseed-lunar/xa.s12.00.mhz.1974-04-19HR00_evid00142.mseed: module 'obspy' has no attribute 'signal'
Error processing /kaggle/input/mseed-lunar/xa.s12.00.mhz.1970-05-25HR00_evid00029.mseed: m

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session